# IMPORTS

In [2]:
import numpy as np
from scipy import signal
from sklearn.linear_model import LinearRegression
import cv2

# FRACTAL TRANSFORMATION

In [3]:
def fractalTrans(img):
    M, N = img.shape[:2]

    if img.ndim == 3:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    img = img.astype(np.float64)
    rmax = 7

    B = {}
    for r in range(2, rmax+1):
        mask = signal.gaussian(r, r/2)
        img = signal.convolve2d(img, mask, mode='same')
        rc = lambda x: np.floor(((np.max(x) - np.min(x)) / r)) + 1
        F = signal.correlate(img, np.ones((r, r)), mode='valid')
        B[r] = np.log(F * (rmax**2 / (r**2)))

    i = np.log(np.arange(2, rmax+1))
    Nxx = np.dot(i, i) - (np.sum(i)**2) / (rmax-1)

    FD = np.zeros((M, N))
    for m in range(M):
        for n in range(N):
            fd = [B[7][m, n], B[6][m, n], B[5][m, n], B[4][m, n], B[3][m, n], B[2][m, n]]
            Nxy = np.dot(i, fd) - (np.sum(i) * np.sum(fd)) / (rmax-1)
            FD[m, n] = Nxy / Nxx

    FD2 = np.maximum(0, FD)
    FDB = (FD2 - np.min(FD2)) / (np.max(FD2) - np.min(FD2))

    return FDB, FD

# MAPPING

In [4]:
def getmapping(samples, mappingtype):    
    table = np.arange(2**samples)
    newMax = 0  # number of patterns in the resulting LBP code
    index = 0

    if mappingtype == 'u2':  # Uniform 2
        newMax = samples * (samples-1) + 3
        for i in range(2**samples):
            j = ((i << 1) | 1) & (2**samples-1)  # rotate left
            numt = np.sum(np.bitwise_xor(i, j) & (2**np.arange(samples)))  # number of 1->0 and 0->1 transitions in binary string x is equal to the number of 1-bits in XOR(x,Rotate left(x))
            if numt <= 2:
                table[i] = index
                index += 1
            else:
                table[i] = newMax - 1

    if mappingtype == 'ri':  # Rotation invariant
        tmpMap = np.full((2**samples,), -1)
        for i in range(2**samples):
            rm = i
            r = i
            for j in range(1, samples):
                r = ((r << 1) | 1) & (2**samples-1)  # rotate left
                if r < rm:
                    rm = r
            if tmpMap[rm] < 0:
                tmpMap[rm] = newMax
                newMax += 1
            table[i] = tmpMap[rm]

    if mappingtype == 'riu2':  # Uniform & Rotation invariant
        newMax = samples + 2
        for i in range(2**samples):
            j = ((i << 1) | 1) & (2**samples-1)  # rotate left
            numt = np.sum(np.bitwise_xor(i, j) & (2**np.arange(samples)))
            if numt <= 2:
                table[i] = np.sum(i & (2**np.arange(samples)))
            else:
                table[i] = samples + 1

    return {'table': table, 'samples': samples, 'num': newMax}

# LBP

In [ ]:
def lbp(image, radius=1, neighbors=8, mapping=0, mode='h', frac=None):
    d_image = np.double(image)
    if radius == 1 and neighbors is None and mapping is None and mode is None:
    spoints = np.array([[-1, -1], [-1, 0], [-1, 1], [0, -1], [0, 1], [1, -1], [1, 0], [1, 1]])
    neighbors = 8
    mapping = 0
    mode = 'h'

if radius is not None and neighbors is not None and mapping is None and mode is None:
    spoints = np.zeros((neighbors, 2))
    a = 2*np.pi/neighbors
    for i in range(neighbors):
        spoints[i][0] = -radius*np.sin((i-1)*a)
        spoints[i][1] = radius*np.cos((i-1)*a)
    mapping = 0
    mode = 'h'

if radius is not None and neighbors is not None and mapping is not None and mode is None:
    spoints = np.zeros((neighbors, 2))
    a = 2*np.pi/neighbors
    for i in range(neighbors):
        spoints[i][0] = -radius*np.sin((i-1)*a)
        spoints[i][1] = radius*np.cos((i-1)*a)
    mode = 'h'

if spoints is not None and mapping is None and mode is None:
    neighbors = spoints.shape[0]
    mapping = 0
    mode = 'h'

if spoints is not None and mapping is not None and mode is None:
    neighbors = spoints.shape[0]
    mode = 'h'

ysize, xsize = image.shape

... 

# FWLB

In [ ]:
def FWLBP(path_image):
    img = cv2.imread(path_image, cv2.IMREAD_GRAYSCALE)
    
    im = np.float32(img)
    FRAC = fractalTrans(im)

    Gray = np.double(img)
    Gray = (Gray - np.mean(Gray)) / np.std(Gray) * 20 + 128

    patternMappingriu2 = getmapping(8, 'x')

    h11 = np.concatenate((lbpfrac(Gray, 1, 8, patternMappingriu2, 'nh', FRAC),
                         lbpfrac(Gray, 2, 8, patternMappingriu2, 'nh', FRAC),
                         lbpfrac(Gray, 3, 8, patternMappingriu2, 'nh', FRAC)))

    h11 = h11 / np.sum(h11)
    return h11